In [14]:
# Setup directories
from pathlib import Path
basedir = Path().absolute()
libdir = basedir.parent.parent.parent

# Other imports
import pandas as pd
import numpy as np
from datetime import datetime

from ioos_qc.plotting import bokeh_multi_plot

from bokeh import plotting
from bokeh.layouts import gridplot
plotting.output_notebook()

Loading BokehJS ...

In [15]:
# Install QC library
#!pip install git+git://github.com/ioos/ioos_qc.git

# # Alternative installation (install specific branch):
# !pip uninstall -y ioos_qc
# !pip install git+git://github.com/ioos/ioos_qc.git@new_configs

# # Alternative installation (run with local updates):
# !pip uninstall -y ioos_qc
# import sys
# sys.path.append(str(libdir))

## Configuration

In [16]:
erddap_server = 'https://ferret.pmel.noaa.gov/pmel/erddap'
dataset_id = 'sd1035_2019'

## Get data from ERDDAP as an xarray object

In [143]:
from erddapy import ERDDAP
e = ERDDAP(
  server=erddap_server,
  protocol='tabledap',
)
e.response = 'csv'
e.dataset_id = dataset_id

In [144]:
ds = e.to_xarray()
ds

<xarray.Dataset>
Dimensions:            (obs: 189492, trajectory: 1)
Coordinates:
  * trajectory         (trajectory) object '1035.0'
    time               (obs) datetime64[ns] ...
    latitude           (obs) float64 ...
    longitude          (obs) float64 ...
Dimensions without coordinates: obs
Data variables:
    rowSize            (trajectory) int32 ...
    TEMP_AIR_MEAN      (obs) float64 ...
    RH_MEAN            (obs) float64 ...
    BARO_PRES_MEAN     (obs) float64 ...
    TEMP_CTD_RBR_MEAN  (obs) float64 ...
    wind_speed         (obs) float64 ...
    wind_dir           (obs) float64 ...
    SAL_SBE37_MEAN     (obs) float64 ...
Attributes:
    area:                      Arctic Ocean
    cdm_data_type:             Trajectory
    cdm_trajectory_variables:  trajectory
    contributor_email:         heather.tabisola@noaa.gov; oar.pmel.data@noaa.gov
    contributor_name:          Heather Tabisola; PMEL Data Integration Group
    contributor_role:          PMEL Mission Manager; PMEL Data Integration Group
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             support@saildrone.com
    creator_name:              Saildrone
    creator_url:               http://saildrone.com/
    data_mode:                 delay-mode
    date_created:              2019-09-22T10:30:36.948664Z
    date_modified:             2020-02-19T23:05:31Z
    description:               Saildrone NetCDF Format
    drone_id:                  1035
    Easternmost_Easting:       -151.2749056
    featureType:               Trajectory
    funding_source:            NOAA/OAR & OMAO
    geospatial_lat_max:        75.3049536
    geospatial_lat_min:        53.8447616
    geospatial_lat_units:      degrees_north
    geospatial_lon_max:        -151.2749056
    geospatial_lon_min:        -168.9577344
    geospatial_lon_units:      degrees_east
    history:                   2020-02-19T23:05:31Z metadata updated by Scien...
    id:                        sd1035_2019_e698_69e3_e309
    infoUrl:                   https://www.pmel.noaa.gov/
    institution:               NOAA/PMEL; NOAA/AFSC; University of Washington
    interval:                  60 s
    keywords:                  acoustic, active, adcp, afsc, air, air_pressur...
    keywords_vocabulary:       GCMD Science Keywords
    license:                   These data are made freely available without r...
    mission:                   NOAA/PMEL 2019 Arctic Saildrone Mission
    naming_authority:          com.saildrone
    netcdf_version:            4.4.1.1
    Northernmost_Northing:     75.3049536
    platform:                  Saildrone
    project:                   NOAA/PMEL 2019 Arctic Saildrone Mission
    qc_manual:                 No QC performed
    sourceUrl:                 (local files)
    Southernmost_Northing:     53.8447616
    standard_name_vocabulary:  CF Standard Name Table v58
    subsetVariables:           trajectory
    summary:                   Six saildrones (sd-1033, sd-1034, sd-1035, sd-...
    time_coverage_duration:    PT1H59M59S
    time_coverage_end:         2019-09-24T13:59:00Z
    time_coverage_resolution:  PT1M
    time_coverage_start:       2019-05-14T22:00:00Z
    title:                     NOAA/PMEL 2019 Arctic Saildrone Mission, drone...
    Westernmost_Easting:       -168.9577344

## Generate a QC configuration for each variable

In [148]:
# Dataset level metadata to drive climatology extraction
min_t = str(ds.time.min().dt.floor("D").dt.strftime("%Y-%m-%d").data)
max_t = str(ds.time.max().dt.ceil("D").dt.strftime("%Y-%m-%d").data)
min_x = float(ds.longitude.min().data)
min_y = float(ds.latitude.min().data)
max_x = float(ds.longitude.max().data)
max_y = float(ds.latitude.max().data)
bbox = [min_x, min_y, max_x, max_y]

In [150]:
# Break down variable by standard name
def not_stddev(v):
    return v and not v.endswith(' SD')

air_temp_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='air_temperature')
pressure_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='air_pressure')
humidity_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='relative_humidity')
water_temp_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='sea_water_temperature')
salinity_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='sea_water_practical_salinity')
uwind_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='eastward_wind')
vwind_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='northward_wind')

In [158]:
# Configure how each variable's config will be generated
default_config = {
    "bbox": bbox,
    "start_time": min_t,
    "end_time": max_t, 
    "tests": {
        "spike_test": {
            "suspect_min": "1",
            "suspect_max": "( 1 + 2 )",
            "fail_min": "3 * 2 - 6",
            "fail_max": "3 * mean + std / ( max * min )"
        },
        "gross_range_test": {
            "suspect_min": "min - std * 2",
            "suspect_max": "max + std / 2",
            "fail_min": "mean * std",
            "fail_max": "mean / std"
        }
    }
}
default_config

{'bbox': [-168.9577344, 53.8447616, -151.2749056, 75.3049536],
 'start_time': '2019-05-14',
 'end_time': '2019-09-25',
 'tests': {'spike_test': {'suspect_min': '1',
   'suspect_max': '( 1 + 2 )',
   'fail_min': '3 * 2 - 6',
   'fail_max': '3 * mean + std / ( max * min )'},
  'gross_range_test': {'suspect_min': 'min - std * 2',
   'suspect_max': 'max + std / 2',
   'fail_min': 'mean * std',
   'fail_max': 'mean / std'}}}

In [159]:
# Construct the inputs
qc_variable_config = {
    
}

In [161]:
# Generate climatology configs
from ioos_qc.config_creator import CreatorConfig, QcConfigCreator, QcVariableConfig

creator_config = {
    "datasets": [
        {
            "name": "ocean_atlas",
            "file_path": "assets/ocean_atlas.nc",
            "variables": {
                "o2": "o_an",
                "salinity": "s_an",
                "temperature": "t_an"
            },
            "3d": "depth"
        },
        {
            "name": "narr",
            "file_path": "assets/narr.nc",
            "variables": {
                "air": "air",
                "pres": "slp",
                "rhum": "rhum",
                "uwnd": "uwnd",
                "vwnd": "vwnd"
            }
        }
    ]
}
cc = CreatorConfig(creator_config)
qccc = QcConfigCreator(cc)

ModuleNotFoundError: No module named 'ioos_qc.config_creator'

In [ ]:
# Generate ioos_qc configs
for v in 
config = qccc(vc)